In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ WARNING: GPU not detected. Enable GPU in Runtime settings!")

## Bước 2: Clone Repository (nếu chưa upload)

In [ ]:
# Option A: Clone từ GitHub (nếu bạn đã push code lên)
# !git clone https://github.com/YOUR_USERNAME/RAG_best_practices.git
# %cd RAG_best_practices

# Option B: Upload từ local
# 1. Zip folder RAG_best_practices trên máy local
# 2. Upload lên Colab bằng cách chạy cell dưới:

from google.colab import files
import zipfile
import os

print("Upload file RAG_best_practices.zip của bạn...")
uploaded = files.upload()

# Unzip
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"Extracted {filename}")

# Change to project directory
%cd RAG_best_practices

## Bước 3: Cài đặt Dependencies

In [ ]:
# Update requirements.txt để dùng faiss-gpu thay vì faiss-cpu
!pip install -q datasets==2.19.1
!pip install -q faiss-gpu  # Dùng GPU version
!pip install -q gensim==4.3.3
!pip install -q hqq==0.2.2
!pip install -q mauve-text==0.4.0
!pip install -q numpy==1.26.4
!pip install -q pandas==2.2.3
!pip install -q rouge-score==0.1.2
!pip install -q safetensors==0.4.5
!pip install -q scikit-learn==1.5.2
!pip install -q sentence-transformers==3.3.0
!pip install -q spacy==3.7.6
!pip install -q tqdm==4.67.0
!pip install -q transformers==4.46.2
!pip install -q accelerate  # Để dùng device_map="auto"
!pip install -q bitsandbytes  # Để quantization

# Download spacy model
!python -m spacy download en_core_web_sm

print("✅ Installation complete!")

## Bước 4: Download Resources (nếu chưa có trong zip)

Nếu bạn chưa upload resources/ folder, download từ Google Drive:

In [ ]:
import os

if not os.path.exists('resources'):
    print("Downloading resources from Google Drive...")
    !gdown --folder https://drive.google.com/drive/folders/1_-2PHI0-Wz1VjnW5Yvy5Ne9C7mMWk1nf
    # Hoặc mount Google Drive của bạn:
    # from google.colab import drive
    # drive.mount('/content/drive')
    # !cp -r /content/drive/MyDrive/RAG_resources ./resources
else:
    print("✅ Resources folder already exists")

## Bước 5: Clone Mixtral-Offloading (cho Mixtral-8x7B)

In [ ]:
if not os.path.exists('mixtral-offloading'):
    !git clone https://github.com/dvmazur/mixtral-offloading.git
    print("✅ Mixtral-offloading cloned")
else:
    print("✅ Mixtral-offloading already exists")

## Bước 6: Kiểm tra cấu trúc project

In [ ]:
!ls -la
print("\n📁 Model folder:")
!ls model/
print("\n📁 Resources folder:")
!ls resources/

## Bước 7: Chạy Evaluation

### Option 1: Chạy với cấu hình mặc định

In [ ]:
!python evaluation.py --dataset truthfulqa --output-dir outputs --seed 42

### Option 2: Chạy trong notebook (để debug dễ hơn)

In [ ]:
import sys
import os

# Disable TensorFlow warnings
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import modules
from model.index_builder import IndexBuilder
from model.language_model import LanguageModel
from model.model_loader import ModelLoader
from model.rag import RAG
from model.retriever import Retriever
from config import configs_run1, configs_run2

import pandas as pd
import numpy as np
from datasets import load_dataset

print("✅ Imports successful!")

In [ ]:
# Load test data
truthful_qa = load_dataset("truthful_qa", "generation", split='validation').to_pandas()
test_data = truthful_qa[['question', 'best_answer', 'correct_answers', 'incorrect_answers']].copy()
test_data['correct_answers'] = test_data['correct_answers'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else [x])
test_data['correct_answers'] = test_data['correct_answers'].apply(lambda x: [i for i in x if i])
test_data['incorrect_answers'] = test_data['incorrect_answers'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else [x])
test_data['incorrect_answers'] = test_data['incorrect_answers'].apply(lambda x: [i for i in x if i])
test_data['best_answer'] = test_data['best_answer'].apply(lambda x: [x] if x else [])
test_data = test_data[(test_data['correct_answers'].apply(len) > 1) & (test_data['incorrect_answers'].apply(len) > 1)]
test_data = test_data.reset_index(drop=True)

print(f"Test data size: {len(test_data)}")
print(test_data.head())

## Bước 8: Xem Results

In [ ]:
# Sau khi chạy xong, xem results
import glob

result_files = glob.glob('outputs/*/*.pkl')
print(f"Found {len(result_files)} result files:")
for f in result_files:
    print(f"  - {f}")

# Load và xem một result
if result_files:
    df = pd.read_pickle(result_files[0])
    print(f"\nSample from {result_files[0]}:")
    print(df.head())

## Bước 9: Download Results về máy

In [ ]:
from google.colab import files
import zipfile

# Zip outputs folder
!zip -r outputs.zip outputs/

# Download
files.download('outputs.zip')

print("✅ Results downloaded!")